In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)

### Hack window for playing with ideas

In [2]:
'''
fi = FileInfo('ext')
fi.textLineCount = 1644
plusCount = 29
minusCount = 0
n = plusCount * 1.0
print(n)
n = n / (plusCount + minusCount)
print(n)
print(fi.textLineCount)
n = n * (1.0 * fi.textLineCount)
print(n)
fi.insertCount = int(fi.textLineCount * ((plusCount * 1.0) / (plusCount + minusCount)))
fi.deleteCount = fi.textLineCount - fi.insertCount
print(fi.insertCount, ',', fi.deleteCount)
'''
0

0

### Main Entry Point

In [3]:
'''
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_log_parser

fileList = []
addFiles(fileList, '/home/matt/Projects/Web3HackerNetwork/data/github');
print('Found '+str(len(fileList))+' files');
root = '/home/matt/Projects/Web3HackerNetwork/data/samples'
log = open(root+'/commitScan.log', 'w')
for fileClass in fileList:
    linecount = 0
    
    if (fileClass.fileName != 'commit-stat.log'):
        #print('Skipping '+fileClass.fileName)
        a = 0
    else:
        fq = fileClass.fullyQualified
        print('Processing ',fq)
        tsvFileName = str(fq)+'.json'
        result = open(tsvFileName, 'w')

        reqSet = StatRequirementSet()

        with open(str(fileClass.fullyQualified), 'r') as file:
            log.write('Reading file: '+fileClass.fullyQualified+'\n')

            for line in file:
                linecount += 1
                reqSet.testline(line);
            file.close()
            
        result.write(json.dumps(reqSet.resultArray, indent=2))
        reqSet.resultArray.clear()
        nBytes = result.tell()
        result.close();
        if (nBytes < 1):
            os.remove(tsvFileName)


        print(str(fileClass.fileName)+' : '+str(linecount)+' lines')
'''
print('All done!')


All done!


### Testing time-zone aware date comparisons

In [4]:
from datetime import datetime as datingdays
from pytz import timezone
last_date = datingdays.fromisoformat('1972-12-26T03:23:01.123456-07:00')
nowness = datingdays.now(timezone('US/Arizona'))
print(nowness - last_date)


18037 days, 9:06:34.780126


### Clone/Pull all of the repositories listed in input file

In [5]:
import os
from git import Repo, Git

with open('./new_repo.log', 'r') as r:
    for l in r.readlines():
        key = l[:-1] #Strip off carriage return
        s = key.split('/')
        owner = s[0]
        repo_name = s[1]
        full_name = key
        print('Processing', owner, repo_name)
        repo_base_dir = './repos'
        repo_path = repo_base_dir+'/'+key

        if (os.path.isdir(repo_base_dir) == False):
            print('######### Cannot find '+repo_base_dir+'  Creating it!')
            os.makedirs(repo_base_dir)
        if (os.path.isdir(repo_base_dir+"/"+owner) == False):
            os.makedirs(repo_base_dir+"/"+owner)
        url = 'https://github.com/'+owner+'/'+repo_name+'.git'
        if (os.path.isdir(repo_path) == False):
            Repo.clone_from(url, repo_path)
        else:
            rp = Repo(repo_path)
            remote = rp.remote()
            remote.pull()

Processing adbenitez mailman
Processing akintu akintu
Processing bbci wyrm
Processing berinhard pyp5js
Processing bhrutledge jahhills.com
Processing Biohazard1976 pi
Processing Biohazard1976 pip
Processing bskinn cpython-release-feed
Processing bskinn flake8-bot
Processing ClearcodeHQ pytest-postgresql
Processing controversial luk.ke
Processing Convex-Dev convex
Processing datacraft-dsc starfish-py
Processing DataDog integrations-core
Processing datalatte-ai Datalatte-marketplace
Processing deltaDAO market-foreverontheblockchain
Processing deltaDAO mvg-portal
Processing deltaDAO portal-euprogigant
Processing deltaDAO portal-safeFBDC
Processing deltaDAO portal-udl
Processing DiddiLeija diddiparser2
Processing DiddiLeija text_formatter
Processing drf-forms drf-schema-adapter
Processing evemorgen SMaDA-AGH
Processing flathub com.valvesoftware.Steam
Processing floss50 keeper-experimental
Processing floss50 ocean-explorer
Processing FMenhorn BGCEGit
Processing fnreality asmgraph
Processing 

### Better way of running git and gather statistics

In [14]:
import git
import json
from datetime import datetime as datingdays
from pytz import timezone
from pydriller import Repository
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_log_parser
from os import listdir
from os.path import isfile, join, isdir

last_date = datingdays.fromisoformat('1972-12-26T03:23:01.123456-07:00')

def introspect(obj):
  for func in [type, id, dir, vars, callable]:
        print("%s(%s):\t\t%s" % (func.__name__, introspect.__code__.co_varnames[0], func(obj)))

class CommitStats():
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=2)    
    def __init__(self):
        self.commit = ''
        self.commit_timestamp = ''
        self.author_name = ''
        self.author_email = ''
        self.file_stats_dic = {}
    def determine_file_type(self, file_name):
        retVal = 'blank'
        if file_name is not None and len(file_name) > 0:
            s = file_name.split('.')
            if len(s) > 1:
                retVal = s[len(s)-1]
        return retVal
    def process_commit(self, c):
        statPerf = 0
        base = datingdays.now().timestamp()
        if c.author is None or c.author.name is None or c.author.email is None:
            print('No author found in commit: ', c.hexsha)
            if c.parents is not None and len(c.parents) > 0:
                print('Trying recursive for ', c.parents[0].hexsha)
                process_commit(c.parents[0])
        else:
            self.commit = c.hash
            self.author_name = c.author.name
            self.author_email = c.author.email
            self.commit_timestamp = datingdays.isoformat(c.committer_date)
            for stat in c.modified_files:
                file_type = self.determine_file_type(stat.filename)
                if file_type not in self.file_stats_dic.keys():
                    self.file_stats_dic[file_type] = {}
                    self.file_stats_dic[file_type]['file_count'] = 0
                    self.file_stats_dic[file_type]['insert_count'] = 0
                    self.file_stats_dic[file_type]['delete_count'] = 0
                self.file_stats_dic[file_type]['file_count'] += 1
                i = stat.added_lines
                d = stat.deleted_lines
                if i == 0 and d == 0:
                    pass
#                    print('Working dir? '+repo.working_dir)
                else:
                    self.file_stats_dic[file_type]['insert_count'] += i
                    self.file_stats_dic[file_type]['delete_count'] += d
                

file_dic = {}
user_map = {}
print(datingdays.now())
master_map = {}
with open('./new_repo.log', 'r') as r:
    for l in r.readlines():
        base_map = {}
        key = l[:-1] #Strip off carriage return
        s = key.split('/')
        owner = s[0]
        repo_name = s[1]
        full_name = key
        repo_base_dir = './repos'
        repo_path = repo_base_dir+'/'+key
        
        cnt = 0
        for commit in Repository(repo_path).traverse_commits():
            cnt += 1
            if cnt % 100 == 0:
                print(cnt,'Processing', owner, repo_name, commit.author.name, commit.author_date, commit.hash)
            obj = CommitStats()
            obj.process_commit(commit)
            base_map[obj.commit] = obj
        print(cnt, 'commits processed total')
        with open(repo_base_dir+'/commit_stat_log.json', 'w') as w:
            w.write(json.dumps(base_map, default=lambda o: o.__dict__, sort_keys=True, indent=2))
        master_map[key] = base_map
        
with open('./master_commit_stat_log.json', 'w') as w:
    w.write(json.dumps(master_map, default=lambda o: o.__dict__, sort_keys=True, indent=2))
    
'''    
    d = c.committed_datetime
    print(d)
    if d > last_date:
        last_date = d;
#    print(c.stats.total)
    for f in c.stats.files:
        stat = c.stats.files[f]
        print(f, stat['insertions'], stat['deletions'], stat['lines'])
    
print('Latest date: ', last_date)  
'''
print('done')

2022-05-15 12:49:37.250748
100 Processing adbenitez mailman mailman 1998-03-05 19:59:09+00:00 98cf560f0161f8d9ee94a14d0ae6a49b3169cd5e
200 Processing adbenitez mailman mailman 1998-03-24 04:49:41+00:00 bc6324353abe0ac5ebf1c566a3c596fd70aecff0
300 Processing adbenitez mailman mailman 1998-04-03 00:19:47+00:00 14e2a06ac6eaaf515f28422c43c785c68686801c
400 Processing adbenitez mailman mailman 1998-04-11 05:27:10+00:00 b8e21cfaf425f8f2de3c1cd740395859d25ad3b5
500 Processing adbenitez mailman klm 1998-05-24 13:44:14+00:00 fc6e55c9a4ce25f11eb6fa12be8d362c419a7dbc
600 Processing adbenitez mailman viega 1998-05-30 06:17:08+00:00 310c4062beee66bb8e3854ead7fa80d9b560a037
700 Processing adbenitez mailman viega 1998-06-14 00:58:33+00:00 8dfd4643c7d8216cfbf6e180ed7b868715b9e775
800 Processing adbenitez mailman klm 1998-07-13 18:19:35+00:00 49c76c3514fa4d5f86c59f57cbab9cf8c5bce4b1
900 Processing adbenitez mailman bwarsaw 1998-08-05 15:44:16+00:00 accc9ae28d977bd3bfbb47d9e71f94f1323fbcc4
1000 Processi

KeyboardInterrupt: 